# Title

## Introduction

1. background on the topic
2. **state the question you will try to answer**
    - one or more random variables
    - spread across two or more categories that are interesting to compare
    - of the response variable, identify one location parameter (mean, median, quantile, etc.) and one scale parameter (standard deviation, inter-quartile range, etc.) that would be useful in answering your question. 
3. Identify and describe the dataset that will be used to answer the question. 
4. at least two scientific publications

## Preliminary Results

- Demonstrate that the dataset can be read from the web into R.
- Clean and wrangle your data into a tidy format.
- Plot the relevant raw data, tailoring your plot in a way that addresses your question.
- Compute estimates of the parameter you identified across your groups. 
- Present this in a table. If relevant, include these estimates in your plot.

In [2]:
library(tidyverse)

NameError: name 'library' is not defined

In [3]:
raw_leaf <- read_csv("data/leaf.csv", show_col_types = FALSE, col_names = c(
    "Class", "Specimen_Number", "Eccentricity", "Aspect_Ratio", 
    "Elongation","Solidity", "Stochastic_Convexity","Isoperimetric_Factor",
    "Maximal_Indentation_Depth", "Lobedness", "Average_Intensity", "Average_Contrast",
    "Smoothness", "Third_moment", "Uniformity", "Entropy"))
head(raw_leaf)

NameError: name 'raw_leaf' is not defined

## Methods: Plan

## References